In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [53]:
df = pd.read_csv('recipe_2.16_삭제본.csv')

In [74]:
# 특징과 레이블 추출
X = df['RCP_PARTS_DTLS']
y = df[['INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT']]    

0     연두부 75g, 칵테일새우 20g, 달걀 30g, 생크림 13g, 설탕 5g, 무염...
1     조선부추 50g, 날콩가루 7g, 저염간장 3g, 다진 대파 5g, 다진 마늘 2g...
2     방울토마토 150g, 양파 10g, 부추 10g, 고춧가루 4g, 멸치액젓 3g, ...
3                    오이 70g, 다진 땅콩 10g, 순두부 40g, 사과 50g
4     북어채 25g, 새우 10g, 사과 30g, 양파 40g, 표고버섯 20g, 물 300g
5     황태 15g, 콩나물 30g, 무 30g, 저염된장 10g, 물 300g 청양고추 ...
6     두부 20g, 애느타리버섯 20g, 감자 10g, 양파 10g, 대파 10g, 된장...
7     국물용 멸치 5g, 다시마 5g 양파 10g, , 국간장 5g, 물 300g, 청경...
8     치커리 30g, 적양배추 15g, 양파 10g, 당근 5g,\r\n 올리브유 10g...
9     딸기 70g, 플레인요거트 85g, 양상추 70g, 메추리알 30g, 블루베리 15...
10    감자 80g, 아몬드 2g, 설탕 2g, 크랜베리 3g, 치커리 5g,\r\n 시금...
11    새송이버섯 70g, 올리브유 10g, 치커리 10g, 연두부 30g, 다진 양파 1...
12    가지 20g, 호박 50g, 새송이버섯 15g, 양파 15g, 발사믹크레마 15g,...
13    브로콜리 40g, 컬리플라워 40g, 적양파 10g, 강낭콩 5g, 건포도 5알 호...
14                                    당근 100g, 오렌지 100g
15    소면 160g, 저염 소금 4g, 식초 8g, 석류 200g, 잣 4g, 아몬드 4...
16    실곤약 440g, 검은콩 70g, 흑임자 6g, 호두 6g, 오이 20g, 토마토 ...
17    만두피 4g, 브로콜리 100g, 숙주 100g, 표고버섯 20g, 닭 가슴

In [59]:
# 각 행의 최대 열 수 확인
max_cols = max(X.apply(len))

# 빈 문자열 및 빈 리스트가 있는 행 제거
X = X[(X.astype(str) != '') & (X.apply(len) > 0)]

# 각 행의 열 수를 최대 열 수로 맞춤
X = X.apply(lambda x: x + [0.0] * (max_cols - len(x)))

# 재료를 부동 소수점으로 변환
X = X.apply(lambda x: [float(i.split(' ')[-1][:-1]) if isinstance(i, str) and i.split(' ')[-1][:-1] else 0.0 for i in x])

# 각 행의 길이를 동일하게 맞추기 위해 패딩된 배열을 사용
X_padded = np.array(X.tolist())

# 결과 확인
print(X_padded.shape)

(50, 226)


In [62]:
y = np.random.rand(X_padded.shape[0], 4)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [63]:
# 데이터 표준화 (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [65]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(4)  # 출력 레이어: INFO_ENG, INFO_CAR, INFO_PRO, INFO_FAT에 해당하는 4개의 뉴런
])

In [67]:
model.compile(optimizer='adam', loss='mse')

In [69]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 0s 63ms/step - loss: 0.3122 - val_loss: 0.2655
Epoch 2/50
1/1 [==============================] - 0s 23ms/step - loss: 0.3113 - val_loss: 0.2647
Epoch 3/50
1/1 [==============================] - 0s 23ms/step - loss: 0.3104 - val_loss: 0.2639
Epoch 4/50
1/1 [==============================] - 0s 24ms/step - loss: 0.3095 - val_loss: 0.2631
Epoch 5/50
1/1 [==============================] - 0s 23ms/step - loss: 0.3086 - val_loss: 0.2623
Epoch 6/50
1/1 [==============================] - 0s 25ms/step - loss: 0.3077 - val_loss: 0.2616
Epoch 7/50
1/1 [==============================] - 0s 24ms/step - loss: 0.3068 - val_loss: 0.2608
Epoch 8/50
1/1 [==============================] - 0s 24ms/step - loss: 0.3059 - val_loss: 0.2600
Epoch 9/50
1/1 [==============================] - 0s 23ms/step - loss: 0.3050 - val_loss: 0.2592
Epoch 10/50
1/1 [==============================] - 0s 24ms/step - loss: 0.3041 - val_loss: 0.2584
Epoch 11/50
1/1 [============

In [73]:
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')

1/1 [==============================] - 0s 17ms/step
Mean Squared Error on Test Set: 0.2555803725284067


In [72]:
# 새로운 테스트 케이스 예제
new_test_case = np.array([[...]])  # 적절한 데이터로 채워넣어야 함

# 예측
predictions = model.predict(new_test_case)

# 결과 확인
print("Predictions:", predictions)


Mean Squared Error on Test Set: 0.2555803725284067
